In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
import duckdb

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
def transform_match_details(df_combined: pd.DataFrame) -> pd.DataFrame:
    cols_to_drop = []
    for col in df_combined.columns:
        if 'info.registry.' in col or 'info.players.' in col:
            cols_to_drop.append(col)
    df_combined.drop(cols_to_drop, axis=1, inplace=True)
    df_combined.columns = [col[5:] if "info." in col else col for col in df_combined.columns]
    df_combined['date'] = df_combined['dates'].apply(lambda x: (x[0]))
    df_match_level_info = df_combined.copy()
    df_match_level_info.drop('innings', axis=1, inplace=True)
    df_match_level_info['officials.umpires'] = df_match_level_info['officials.umpires'].apply(lambda x: ','.join(map(str, x)))
    df_match_level_info[['Umpire1', 'Umpire2']] = df_match_level_info['officials.umpires'].str.split(',', expand=True)
    try:
        df_match_level_info['officials.tv_umpires'] = df_match_level_info['officials.tv_umpires'].apply(lambda x: ','.join(map(str, x)))
        df_match_level_info['officials.reserve_umpires'] = df_match_level_info['officials.reserve_umpires'].apply(lambda x: ','.join(map(str, x)))
    except Exception as e:
        print(e)
        print(df_match_level_info['officials.reserve_umpires'])
    df_match_level_info['officials.match_referees'] = df_match_level_info['officials.match_referees'].apply(lambda x: ','.join(map(str, x)))
    df_match_level_info['teams'] = df_match_level_info['teams'].apply(lambda x: ','.join(map(str, x)))
    df_match_level_info[['Team1', 'Team2']] = df_match_level_info['teams'].str.split(',', expand=True)
    df_match_level_info['event.match_number'].fillna(df_match_level_info['event.stage'], inplace=True)
    df_match_level_info['event.match_number'] = df_match_level_info['event.match_number'].astype(str).str.rstrip('.0')
    df_match_level_info['player_of_match'] = df_match_level_info['player_of_match'].fillna("NA").apply(lambda x: (x[0]))
    df_match_level_info.drop(['meta.data_version', 'meta.created', 'meta.revision', 'dates', 'event.stage'], axis=1, inplace=True)
    df_match_level_info['outcome_type_1'] = df_match_level_info["outcome.by.runs"].apply(lambda x: x if pd.isna(x) else "Runs")
    df_match_level_info['outcome_type_2'] = df_match_level_info["outcome.by.wickets"].apply(lambda x: x if pd.isna(x) else "Wickets")
    df_match_level_info['outcome_type'] = df_match_level_info['outcome_type_1'].fillna(df_match_level_info['outcome_type_2'])
    df_match_level_info.drop(['officials.umpires', 'outcome_type_1', 'outcome_type_2', 'teams'], axis=1, inplace=True)
    df_match_level_info.columns = [col.replace(".", "_") for col in df_match_level_info.columns]
    df_match_level_info = df_match_level_info[[ 'event_match_number', 'date', 'event_name', 'season', 'team_type', 'gender', 'match_type', 'overs', 'balls_per_over', 'Team1', 'Team2',
    'toss_decision', 'toss_winner', 'venue','city','outcome_type','outcome_by_wickets', 'outcome_by_runs','outcome_winner','outcome_method', 'outcome_result', 'player_of_match', 'Umpire1', 'Umpire2',
    'officials_match_referees', 'officials_reserve_umpires',
    'officials_tv_umpires']]
    return df_match_level_info

In [ ]:
json_files = glob.glob("./ipl_male_json/" + "*.json")

In [ ]:
%%markdown
## Desired Files

In [ ]:
dataset_list = []
for json_file in json_files:
    json_file_name = os.path.basename(json_file) #1359475
    file_name, file_extension = os.path.splitext(json_file_name)
    if int(file_name) >= 0: #1359475
        try:
            with open(json_file, 'r') as json_file:
                data = json.load(json_file)
                df = pd.json_normalize(data)
                dataset_list.append(df)
        except Exception as e:
            print(f"Error reading {json_file}: {e}")

In [ ]:
df_combined = pd.concat(dataset_list)

In [ ]:
match_fact_table = transform_match_details(df_combined)

In [ ]:
match_fact_table

In [ ]:
match_fact_table.to_csv("ipl_match_data.csv", index=False)

In [ ]:
duckdb.query("""
SELECT venue, CASE WHEN toss_winner == outcome_winner THEN 'TOSS WINNER WINS' ELSE 'TOSS WINNER LOST' END AS drv_winner_details, COUNT(*) as occurances FROM match_fact_table GROUP BY 1,2 ORDER BY 1
""").to_df()

In [ ]:
duckdb.query("""
SELECT outcome_winner, MAX(outcome_by_runs) as result FROM match_fact_table 
WHERE outcome_winner != 'None'
GROUP BY 1 ORDER BY 2 DESC
""").to_df()

In [ ]:
duckdb.query("""
SELECT season, player_of_match, COUNT(*) FROM match_fact_table 
GROUP BY 1,2 
HAVING COUNT(*) >= 2
ORDER BY 3 DESC
""").to_df()

In [ ]:
match_fact_table.columns

In [2]:
df1 = pd.read_csv("ipl_match_data.csv")

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   event_match_number         1030 non-null   object 
 1   date                       1030 non-null   object 
 2   event_name                 1030 non-null   object 
 3   season                     1030 non-null   object 
 4   team_type                  1030 non-null   object 
 5   gender                     1030 non-null   object 
 6   match_type                 1030 non-null   object 
 7   overs                      1030 non-null   int64  
 8   balls_per_over             1030 non-null   int64  
 9   Team1                      1030 non-null   object 
 10  Team2                      1030 non-null   object 
 11  toss_decision              1030 non-null   object 
 12  toss_winner                1030 non-null   object 
 13  venue                      1030 non-null   objec